In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [23]:
def e_s(t):
    e = 0.6108 * np.exp((17.27*t)/(t+237.3))
    e = e*10
    return e
def delta(es, t):
    delt = (4098*es) / (t+237.3)**2
    return delt
def gamma(P):
    gamma = (1930*P) / (2257000*0.622)
    return gamma
def resistencia_aerodinamica(z, d, z0m, z0h, u):
    resistencia = (np.log((z-d)/z0m)
                   * np.log((z-d)/z0h)
                   /(0.4**2 * u)
                  )
    return resistencia
def Epm(delta, Rn, es, e, rs, ra, u, gamma):
    Elv = ((delta*(0.9*Rn) + 1.23*1004*((es-e)/ra))
           / (delta + gamma*(1+(rs/ra)))
          )
    return Elv

In [24]:
datos = pd.read_csv(
    'C:\\Users\\nahue\\Desktop\\Agro 2\\Tp_2\\Ej_3.txt',
    delimiter = ' ',
    decimal = '.'
    )

In [25]:
datos.head(10)

,Década,PP,ETP
0,10/11,40,28.0
1,20/11,75,25.0
2,30/11,10,35.0
3,10/12,15,40.0
4,20/12,0,40.0
5,31/12,10,32.0
6,10/1,50,NaN


In [26]:
kc = 1.25
almacenaje_inicial = 260
capacidad_campo = 300
temperatura = 20
presion_vapor = 17
radiacion_neta = 10
viento = 10/3.6
rugosidad_movimiento = 1.5*10**-2
rugosidad_humedad = 1.5*10**-3
plano_cero = 8*10**-2
resistencia_superficie = 70
z = 2

In [27]:
presion_saturacion = e_s(temperatura)
delta_pm = delta(presion_saturacion, temperatura)
gamma_pm = gamma(1013)
resistencia_pm = resistencia_aerodinamica(z,
                                         plano_cero,
                                         rugosidad_movimiento,
                                         rugosidad_humedad,
                                         viento)
datos['ETP'][6] = Epm(delta_pm, radiacion_neta, presion_saturacion/10,
                      presion_vapor/10, resistencia_superficie, resistencia_pm,
                      viento, gamma_pm
                     ) * 10
datos.head(10)

C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,Década,PP,ETP
0,10/11,40,28.000000
1,20/11,75,25.000000
2,30/11,10,35.000000
3,10/12,15,40.000000
4,20/12,0,40.000000
5,31/12,10,32.000000
6,10/1,50,56.549059


In [28]:
resistencia_pm

78.10742303651055

In [29]:
presion_saturacion

23.38281270927446

In [30]:
delta_pm

1.4474018811241363

In [31]:
gamma_pm

1.3926590656863178

In [32]:
datos['ETR'] = datos['ETP']*kc
datos.head(10)

,Década,PP,ETP,ETR
0,10/11,40,28.000000,35.000000
1,20/11,75,25.000000,31.250000
2,30/11,10,35.000000,43.750000
3,10/12,15,40.000000,50.000000
4,20/12,0,40.000000,50.000000
5,31/12,10,32.000000,40.000000
6,10/1,50,56.549059,70.686323


    El punto de marchitez permanente (PMP) es el contenido de agua mínimo para que las plantas puedan extraer agua del suelo y evapotranspirarla. En este caso se observa que por debajo de d = 100mm en los primeros dos metros, el cultivo deja de evapotranspirar, por lo que estaría en el PMP. Equivale a un tita de 0.05. El agua útil es la diferencia entre la capacidad de campo (CC) y el PMP. Para este caso, el suelo tiene 200mm de agua útil en los primeros 2 metros que equivalen a un tita de 0.1.

In [33]:
datos['balance'] = list(range(7))
datos['balance'][0] = 260 - datos['ETR'][0] + datos['PP'][0]
iterador = list(range(1,7))
for i in iterador:
    datos['balance'][i] = datos['balance'][i-1] - datos['ETR'][i] + datos['PP'][i]
    if datos['balance'][i] > capacidad_campo:
        datos['balance'][i] = capacidad_campo
datos.head(10)

C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Década,PP,ETP,ETR,balance
0,10/11,40,28.000000,35.000000,265
1,20/11,75,25.000000,31.250000,300
2,30/11,10,35.000000,43.750000,266
3,10/12,15,40.000000,50.000000,231
4,20/12,0,40.000000,50.000000,181
5,31/12,10,32.000000,40.000000,151
6,10/1,50,56.549059,70.686323,130


    En la segunda década hubo escorrentía.
    Con un almacenaje menor a 180mm el cultivo entra en estrés hídrico, pues su evapotranspiración real es menor a la máxima. Se puede observaer en el balance calculado que durante las primeras 5 decadas el cultivo se encontró en condiciones óptimas, pero se estresó en las últimas dos.